# Buidling an agent using Langchain

Basic building blocks:
* Tools: Tools in the context of Large Language Models refer to various functionalities and extensions that enhance the capabilities of these models beyond basic text generation. 
    * Langchain default tools
    * Custom tools
* agent: agent llm + prompt + invoke chain
* agent executor

**Ideas**
* what about a summarization tool? I'd like to see if we can 1) extract wiki content and then 2) summarize it
* Better wiki tool: QA retrieval chain: https://python.langchain.com/v0.1/docs/integrations/retrievers/wikipedia/
* this shows how to set up the wikipedia tool as custom tool: https://python.langchain.com/v0.1/docs/modules/tools/
* what is the difference between tools and openai functions


Use cases:
* create an agent that cna combine wikipedia data with relevant real-world information from API's. PRovide comprehensive answers tha encompass historical context and current data (eg weather, )
* create agent that can answer questions about specific topics by searching wiki, use NLP to analyze the articles content, summarize the key info and identify relevant sections, provide clear answers

In [ ]:
#Todo: dependency management

# !pip3 install langchain
# !pip3 install langchain_openai
# !pip3 install langchain_community
# pip install wikipedia
# pip install langchainhub

In [27]:
!pip3 freeze > 'dependencies.txt'

### Exercises overview
1. Familiarize yourself with the default wikipedia tool
        <ol type="a">
        <li>Explore tool parameters</li>
        <li>Run tool and explore output</li>
        </ol>
2. Build your own tool: weather api


## A Langchain default tool: the [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/)


The cell below loads the full wikipedia tool. It makes an API call to Wikipedia using the ``WikipediaAPIWrapper`` and returns a summary of the queried article. ``WikipediaQueryRun`` then wraps this into a ready made tool.

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)#, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### Exercise 1 (a): Familiarize yourself with a default tool

Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

Background: each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [46]:
print("Name: ", wiki_tool.name)

### enter your code here


Name:  wikipedia


In [9]:
# # solution: 
# print("Description: ", wiki_tool.description)
# print("Input argument schema: ", wiki_tool.args)
# print("Return output to user? ", wiki_tool.return_direct)
# print("Metadata: ", wiki_tool.metadata)

#### Exercise 1 (b)

* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

In [ ]:
# enter your code here

In [12]:
# solution
query = 'pyladies'
query = 'ruth bader ginsburg'

print(wiki_tool.run(query))


Page: Ruth Bader Ginsburg
Summary: Joan Ruth Bader Ginsburg ( BAY-dər GHINZ-burg; née Bader; March 15, 1933 – September 18, 2020) was an American lawyer and jurist who served as an associate justice of the Supreme Court of the United States from 1993 until her death in 2020. She was nominated by President Bill Clinton to replace retiring justice Byron White, and at the time was viewed as a moderate consensus-builder. Ginsburg was the first Jewish woman and the second woman to serve on the Court, after Sandra Day O'Connor. During her tenure, Ginsburg authored the majority opinions in cases such as United States v. Virginia (1996), Olmstead v. L.C. (1999), Friends of the Earth, Inc. v. Laidlaw Environmental Services, Inc. (2000), and City of Sherrill v. Oneida Indian Nation of New York (2005). Later in her tenure, Ginsburg received attention for passionate dissents that reflected liberal views of the law. She was popularly dubbed "the Notorious R.B.G.", a moniker she later embraced.
Gins

## Custom tools

You can build your own tools and don't have to rely on default tools. Tools can be built from any function with the LangChain class method ``StructuredTool.from_function()``(see [here](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)). The basic elements are
* The **function** you would like to be executed when the tool is called
* The definition of the **input parameters**
* The tool **description**

The tool description is especially important, since this is what the agent will use to make the deicion if this tool should be used.

Below you see the wikipedia tool, built from the basic elements described above:

In [8]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool. 
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('pyladies'))

Page: PyLadies
Summary: PyLadies is an international mentorship group which focuses on helping more women become active participants in the Python open-source community. It is part of the Python Software Foundation. It was started in Los Angeles in 2011. The mission of the group is to create a diverse Python community through outreach, education, conferences and social gatherings. PyLadies also provides funding for women to attend open source conferences. The aim of PyLadies is increasing the participation of women in computing. PyLadies became a multi-chapter organization with the founding of the Washington, D.C., chapter in August 2011.




#### Exercise 2
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

Task: 
- The tool function is already provided to your. Build the tool by defining the input parameters and the descriptions. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``
- test if the tool gives an output

In [ ]:
import requests # for API calls

# define the function
def extract_city_weather(city:str)->str:
    api_key = 'ZHK4HRAZ3MTNV5FDNQKFYUGL7'

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={api_key}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    # insert your code here

# the tool description
description: str = (
        # insert your code here
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    # insert your code here
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

In [9]:
import requests

# define the function
def extract_city_weather(city:str)->str:
    api_key = 'ZHK4HRAZ3MTNV5FDNQKFYUGL7'

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={api_key}&unitGroup=metric"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="City name")


# the tool description
description: str = (
        "Allows to extract the current temperature in a specific city"
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    func=extract_city_weather,
    name="weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False,
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

Current temperature in Amsterdam: 13.8°C


Combine the individual tools into a list. Your collection of tools is not ready to be used by an agent.

In [10]:

tools = [my_own_wiki_tool, weather_tool]

# Define Agent in 3 lines

In [3]:
# dependencies
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub # for the prompt, we are going to skip this
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent

/Users/maria/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:

# agent llm
key = 'sk-proj-XX'
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=key)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [54]:
# agent definition
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [55]:
# agent_executor.invoke({"input": "who is Ryan Gosling?"})
# agent_executor.invoke({"input": "what is three times 5"})
question = "How old is Ryan gosling?"
question = 'how many children did ruth bader ginsburg have?'
question = 'write me a short summary about ruth bader ginsburg'
question = 'what was ruth bader ginsburg passionate about?'
question = 'who were ruth baeder ginsburgs colleagues at the supreme court?'
question = 'what are great holiday destinations?'
question = 'which city is bigger: Paris or Munich?'
question = 'which city is better for vacation, Paris or Amsterdam?'
question = 'how is the weather where Ryan gosling lives?'

agent_executor.invoke({"input": question})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Ryan Gosling'}`


Page: Ryan Gosling
Summary: Ryan Thomas Gosling ( GOSS-ling; born November 12, 1980) is a Canadian actor. Prominent in both independent films and major studio features, his films have grossed over $2 billion worldwide. Gosling has received various accolades, including a Golden Globe Award, and nominations for three Academy Awards and two British Academy Film Awards.
Gosling rose to prominence aged 13 on Disney Channel's The Mickey Mouse Club (1993–1995), and went on to appear in other family entertainment programs, including Are You Afraid of the Dark? (1995) and Goosebumps (1996). His breakthrough role was that of a Jewish neo-Nazi in The Believer (2001), and he gained stardom in the 2004 romantic drama The Notebook. He starred in the critically acclaimed independent dramas Half Nelson (2006), for which he was nominated for the Academy Award for Best Actor; Lars and the Real Girl (2007), 

{'input': 'how is the weather where Ryan gosling lives?',
 'output': 'The current weather in Los Angeles, where Ryan Gosling lives, is 18.0°C.'}

# What happens under the hood 

An agent is nothing more than a while loop: While the LLM expects a function call to be executed, keep querying the LLM.

In [21]:
from openai import OpenAI
from helper_functions import simple_description_formatter
import json
client = OpenAI(api_key = key)

# format tool description
function_tools = [simple_description_formatter(tool) for tool in tools]

# extract available functions from list of tools
available_functions = {tool.name: tool for tool in tools}

question = "How many people live in Paris?"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": question}
  ]


# first response
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  tools = function_tools,
  messages=messages, 
  )

print(f"AI response: {response}\n")

while response.choices[0].message.tool_calls:
    
    # response message with marameters needed for function call
    response_message = response.choices[0].message
    messages.append(response_message)

    #extract tool calls
    tool_calls = response.choices[0].message.tool_calls

    for tool_call in tool_calls:
        # function name
        function_name = response.choices[0].message.tool_calls[0].function.name
        
        # function arguments
        function_args = json.loads(tool_call.function.arguments)
        print(f'Calling tool "{function_name}" with arguments {function_args}.')

        # execute function call 
        function_response = available_functions[function_name].invoke(function_args)
        print(f'Function call response:\n{function_response}\n')

        # append function response to messages
        messages.append({
            "tool_call_id":tool_call.id, 
            "role": "tool", 
            "name": function_name, 
            "content": function_response
        })
      
        # get a new response from LLM
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          tools = function_tools,
          messages=messages, 
        )
        print(f"AI response: {response}\n")

print("Question: ", question)
print(f"Final response: {response.choices[0].message.content}")


AI response: ChatCompletion(id='chatcmpl-9W701XHjMFBbvg6eAC2GTcBiv4nT6', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FXp8H2ased7d8vD7sTtop9ln', function=Function(arguments='{"query":"Population of Paris"}', name='wikipedia'), type='function')]))], created=1717440429, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=139, total_tokens=155))

Calling tool "wikipedia" with arguments {'query': 'Population of Paris'}.
Function call response:
Page: Demographics of Paris
Summary: The city of Paris (also called the Commune or Department of Paris) had a population of 2,165,423 people within its administrative city limits as of January 1, 2019. It is surrounded by the Paris unité urbaine, or urban area, the most populous urban area in the European Union. I

# Sources
* https://python.langchain.com/docs/modules/agents/quick_start/